In [ ]:
# Verificação do caminho dos arquivos iniciais e funções que irão auxiliar no decorrer do notebook

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
def get_iqr_values(df_in, col_name):
    median = df_in[col_name].median()
    q1 = df_in[col_name].quantile(0.25) # 25% / 1st quartile
    q3 = df_in[col_name].quantile(0.75) # 7th percentile / 3rd quartile
    iqr = q3-q1 #Interquartile range
    minimum  = q1-1.5*iqr # The minimum value or the |- marker in the box plot
    maximum = q3+1.5*iqr # The maximum value or the -| marker in the box plot
    return median, q1, q3, iqr, minimum, maximum
  
def get_iqr_text(df_in, col_name):
    median, q1, q3, iqr, minimum, maximum = get_iqr_values(df_in, col_name)
    text = f"median={median:.2f}, q1={q1:.2f}, q3={q3:.2f}, iqr={iqr:.2f}, minimum={minimum:.2f}, maximum={maximum:.2f}"
    return text
  
def remove_outliers(df_in, col_name):
    _, _, _, _, minimum, maximum = get_iqr_values(df_in, col_name)
    df_out = df_in.loc[(df_in[col_name] > minimum) & (df_in[col_name] < maximum)]
    return df_out
  
def count_outliers(df_in, col_name):
    _, _, _, _, minimum, maximum = get_iqr_values(df_in, col_name)
    df_outliers = df_in.loc[(df_in[col_name] <= minimum) | (df_in[col_name] >= maximum)]
    return df_outliers.shape[0]
  
def box_and_whisker(df_in, col_name):
    title = get_iqr_text(df_in, col_name)
    sns.boxplot(df_in[col_name])
    plt.title(title)
    plt.show()
    
# Função para remover os outliers 
def remove_all_outliers(df_in, col_name):
    loop_count = 0
    outlier_count = count_outliers(df_in, col_name)
    while outlier_count > 0:
        loop_count += 1
        if (loop_count > 100):
            break
        df_in = remove_outliers(df_in, col_name)
        outlier_count = count_outliers(df_in, col_name)
    return df_in

In [ ]:
# Importação das principais libs que utiizaremos ao longo do código

import numpy as np
import pandas as pd 
import seaborn as sns
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics
from sklearn.metrics import accuracy_score
import time

import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
# Descompactação das bases usando o ZipFile e criação dos dataframes principais

from zipfile import ZipFile

with ZipFile('../input/walmart-recruiting-store-sales-forecasting/features.csv.zip') as f:
    f.extractall(path='walmart-recruiting-store-sales-forecasting')  
    
features = pd.read_csv("./walmart-recruiting-store-sales-forecasting/features.csv")

with ZipFile('../input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip') as f: 
    f.extractall(path= 'walmart-recruiting-store-sales-forecasting')

submission = pd.read_csv("./walmart-recruiting-store-sales-forecasting/sampleSubmission.csv")
    
with ZipFile('../input/walmart-recruiting-store-sales-forecasting/test.csv.zip') as f: 
    f.extractall(path= 'walmart-recruiting-store-sales-forecasting')

test = pd.read_csv("./walmart-recruiting-store-sales-forecasting/test.csv")

with ZipFile('../input/walmart-recruiting-store-sales-forecasting/train.csv.zip') as f: 
    f.extractall(path= 'walmart-recruiting-store-sales-forecasting')

train = pd.read_csv("./walmart-recruiting-store-sales-forecasting/train.csv")

stores = pd.read_csv("../input/walmart-recruiting-store-sales-forecasting/stores.csv")

# Análise basica dos dataframes

In [ ]:
# Análise básica dos DataFrames

print("features.shape", features.shape)
#print("submission.shape", submission.shape)
print("test.shape", test.shape)
print("train.shape", train.shape)
print("stores.shape", stores.shape)

Inicialmente, podemos ver que os dataframes "submission" e "test" tem a mesma quantidade de linhas. E que o DataFrame Stores, deve ser de cardinalidade 1 para muitos. 

***Features***

In [ ]:
features.head(5)

In [ ]:
features.dtypes

***Submission***

***Test***

In [ ]:
test.head(5)

In [ ]:
test.dtypes

***Train***

In [ ]:
train.head(5)

In [ ]:
train.dtypes

***Stores***

In [ ]:
stores.head(5)

In [ ]:
stores.dtypes

Vamos **juntar** os dataframes de **lojas e features** com os dataframes de **treino e teste**

In [ ]:
df_train = train.merge(stores, how='left').merge(features, how='left')
df_test = test.merge(stores, how='left').merge(features, how='left')

In [ ]:
print("df_train.shape", df_train.shape)
print("train.shape", train.shape)

print("df_test.shape", df_test.shape)
print("test.shape", test.shape) 

# Tratamentos no DataFrame de treino

In [ ]:
#Verificação de valores nulos no dataframe de treino

miss_sum = pd.DataFrame(df_train.isnull().sum().sort_values(ascending=False), columns = ['Total'])
miss_percent = pd.DataFrame((df_train.isnull().sum()/df_train.isnull().count()*100), columns = ['Percentage'])
missfulldf = pd.concat([miss_sum,miss_percent], axis=1)

print(missfulldf[missfulldf['Total']>0].sort_values(by= 'Percentage', ascending= False))
print('\n********TOTALS**********\n',missfulldf[missfulldf['Total']>0].count())

In [ ]:
# Preenchimento dos valores nulos com 0

df_train["MarkDown1"].fillna(0, inplace= True)
df_train["MarkDown2"].fillna(0, inplace= True)
df_train["MarkDown3"].fillna(0, inplace= True)
df_train["MarkDown4"].fillna(0, inplace= True)
df_train["MarkDown5"].fillna(0, inplace= True)

In [ ]:
df_train["Date"]= pd.to_datetime(df_train["Date"])

In [ ]:
# Como a variável target é a predição do volume de vendas por semana, vamos analisar o volume de vendas por semanas do ano

df_train["week"]= df_train.Date.dt.isocalendar().week
df_train["year"]= df_train.Date.dt.year
df_train['Month'] = df_train.Date.dt.month
df_train['Day'] = df_train.Date.dt.day

# Tratamentos no DataFrame de teste

In [ ]:
#Verificação de valores nulos no dataframe de teste

miss_sum = pd.DataFrame(df_test.isnull().sum().sort_values(ascending=False), columns = ['Total'])
miss_percent = pd.DataFrame((df_test.isnull().sum()/df_test.isnull().count()*100), columns = ['Percentage'])
missfulldf = pd.concat([miss_sum,miss_percent], axis=1)

print(missfulldf[missfulldf['Total']>0].sort_values(by= 'Percentage', ascending= False))
print('\n********TOTALS**********\n',missfulldf[missfulldf['Total']>0].count())

Podemos ver que existe uma diferença quando comparamos o dataframe de treino com o dataframe de testes. 
   
    DataFrame de treino, apenas as variáveis de MarkDown vieram com campos nulos. 
    DataFrame de teste temos valores nulos nas variáveis CPI e Unemployment

In [ ]:
# Aplicação dos mesmos passos utilizados no DataFrame de treino

df_test["MarkDown1"].fillna(0, inplace= True)
df_test["MarkDown2"].fillna(0, inplace= True)
df_test["MarkDown3"].fillna(0, inplace= True)
df_test["MarkDown4"].fillna(0, inplace= True)
df_test["MarkDown5"].fillna(0, inplace= True)

df_test["Date"]= pd.to_datetime(df_test["Date"])

df_test["week"]= df_test.Date.dt.isocalendar().week
df_test["year"]= df_test.Date.dt.year
df_test['Month'] = df_test.Date.dt.month
df_test['Day'] = df_test.Date.dt.day

In [ ]:
# Função criada no início do notebook
_= box_and_whisker(df_test, "CPI")

Podemos ver que a varíavel "CPI" está normalmente distribuída. Neste caso, optei preencher os valores nulos com a média. 

In [ ]:
df_test["CPI"].fillna(df_test["CPI"].mean(), inplace= True)

In [ ]:
# Função criada no início do notebook
_= box_and_whisker(df_test, "Unemployment")

Podemos ver que a varíavel "Unemployment" está normalmente distribuída. Neste caso, optei preencher os valores nulos com a média. 

In [ ]:
df_test["Unemployment"].fillna(df_test["Unemployment"].mean(), inplace= True)

In [ ]:
df_train.dtypes

# EDA

Algumas perguntas que podemos fazer para os dados: 

    Quando a temperatura aumenta, as vendas sobem? 
    As vendas semanais costumam aumentar nos feriados? 
    O tipo de loja ou tamanho da loja influencia no volume de vendas?
    Existe algum tipo de sazonalidade de compras?

In [ ]:
# Alteração dos tipos de variáveis

df_train= df_train.astype({"Store":"int32",
                           "Dept":"int32",
                           "Type":"object",
                           "IsHoliday":"bool",
                           "Type":"object",
                           "week":"int16",
                           "year":"int16",
                           "Month":"int16",
                           "Day":"int16"})


df_test= df_test.astype({"Store":"int32",
                         "Dept":"int32",
                         "Type":"object",
                         "IsHoliday":"bool",
                         "Type":"object",
                         "week":"int16",
                         "year":"int16",
                         "Month":"int16",
                         "Day":"int16"})

featdf = pd.DataFrame(df_train.dtypes,columns=['Data_Type'])
nunfeat = list(featdf[featdf['Data_Type']!='object'].index)
catfeat = list(featdf[featdf['Data_Type']=='object'].index)

In [ ]:
df_train[nunfeat].describe().T

In [ ]:
# Vendas por semanas do ano

fig, ax = plt.subplots()

fig.set_size_inches(25, 10)

_ = sns.lineplot(x= "week", 
                 y= "Weekly_Sales", 
                 data= df_train,
                 hue= "year", 
                 ci= None)

plt.show()

De antemão, podemos ver que ocorrem 3 picos de vendas todos os anos, um pico pequeno por volta da 12-14 semana, e dois picos grandes por volta das 46-47 e 51-51.

Quando olhamos as datas, vemos que os maiores picos são o dia de ação de graças e natal. 

In [ ]:
_= sns.catplot(x= "Store", 
               y= "Weekly_Sales", 
               data= df_train,
               kind= "bar",
               hue= "Type", 
               height= 10, 
               aspect= 2)

Podemos ver que as lojas do tipo "A" tendem a ter uma média semanal de vendas maior que as outras. 

In [ ]:
plt.figure(figsize=(16,8))

_= sns.scatterplot(x= "Temperature", 
                   y= "Weekly_Sales", 
                   data= df_train,
                   hue= "Type")

Com o gráfico acima, podemos ver que a média de vendas semanais, tentem a aumentar quando a temperatura esta entre 40 e 60 grau e que tendem a diminuir nos extremos. Porém não aparenta existir alguma correlação forte com esta hipótese. 

In [ ]:
_= sns.catplot(x= "IsHoliday",
               y= "Weekly_Sales",
               data= df_train,
               kind= "violin")

Podemos ver que as vendas semanais nas semanas que tem feriados tendem a ser mais altas do que as que não tem. 

In [ ]:
plt.figure(figsize=(16,8))

_= sns.scatterplot(x= "Size",
                   y= "Weekly_Sales",
                   data= df_train,
                   hue= "Type")

O gráfico acima, mostra que existe uma correlação entre vendas semanais com o tipo de loja e tamanho da loja.

In [ ]:
plt.figure(figsize=(16,8))

_= sns.scatterplot(x= "Dept",
                   y= "Weekly_Sales",
                   data= df_train,
                   hue= "Type")

Podemos ver pelo gráfico acima, que existe alguma correlação entre o volume de vendas semanais com os departamentos. 

In [ ]:
# Correlação para medir a força de correlação linear entre as variáveis

f, ax= plt.subplots(figsize= (15, 10))

corr= df_train[nunfeat].corr(method='pearson')

mask= np.triu(np.ones_like(corr, dtype= bool))

_= sns.heatmap(corr, 
               mask= mask,
               cmap='coolwarm', 
               annot=True, 
               center= 0, 
               fmt= '.3f',
               square= True)

_.set_title('Correlação de Pearson');

Aparentemente, não existe uma correlação muito forte entre entre variável target e outras variáveis.



# Feature Engineering

In [ ]:
# Criaçao das variáveis X e y

X= df_train.drop(columns= ["Date", 
                           "Weekly_Sales"])
                        
y= df_train["Weekly_Sales"]

In [ ]:
featdf = pd.DataFrame(X.dtypes,columns=['Data_Type'])

nunfeat = list(featdf[featdf['Data_Type']!='object'].index)
catfeat = list(featdf[featdf['Data_Type']=='object'].index)
encfeat= ["IsHoliday","Type"]

In [ ]:
#Criação dos dataframes de treino, teste e validação

from sklearn.model_selection import train_test_split

# Dataset de treino
X_train, X_rem, y_train, y_rem= train_test_split(X, y, train_size= 0.8)

#Dataset de validação
X_valid, X_test, y_valid, y_test= train_test_split(X_rem, y_rem, test_size= 0.5)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print(X_valid.shape, y_valid.shape)

In [ ]:
# Criação dos encoders para as variáveis categóricas

import pickle
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse=False, handle_unknown='ignore').fit(X_train[encfeat])
encoded_cols_train = list(enc.get_feature_names(encfeat))

# Gravação do encoder com pickle para podermos persistir o mesmo padrão em futuros dataframes 
filehandler= open("enc.obj", "wb")
pickle.dump(enc, filehandler)
filehandler.close()


In [ ]:
#Persistindo o encoder para X_train, X_test, X_valid

X_train[encoded_cols_train]= enc.transform(X_train[encfeat])
X_train.drop(columns= encfeat, inplace= True)

X_test[encoded_cols_train]= enc.transform(X_test[encfeat])
X_test.drop(columns= encfeat, inplace= True)

X_valid[encoded_cols_train]= enc.transform(X_valid[encfeat])
X_valid.drop(columns= encfeat, inplace= True)

# Treinamento Inicial

In [ ]:
from xgboost import XGBRegressor

model = XGBRegressor(random_state=42, 
                     n_estimators= 25, 
                     max_depth=4)

In [ ]:
%%time

model.fit(X_train, y_train)

In [ ]:
preds = model.predict(X_test)

In [ ]:
# Funçao para calcular o WMAE  

def WMAE(df, targets, predictions):
    weights = df.IsHoliday_True.apply(lambda x: 1 if x==0 else 5)
    return np.round(np.sum(weights*abs(targets-predictions))/(np.sum(weights)), 2)

In [ ]:
#WMAE Inicial

WMAE(X_test, y_test, preds)

In [ ]:
# Verificação da importância das variáveis

plt.figure(figsize=(10,7))
_= sns.barplot(x= "importance",
            y= "feature", 
            data= pd.DataFrame({"feature": X_train.columns, "importance": model.feature_importances_}).sort_values(by= "importance", ascending= False)) 

Vemos que inicialmente, algumas variáveis tiveram pouca importância para predizer as vendas semanais e que as variáveis mais importantes foram o tamanho da loja e o departamento. 

# Ajuste de hiperparâmetros


**** Este passo pode levar muito tempo para ser executado e é necessário que a GPU seja ativada nas preferências de ambiente. 

import optuna

def objective(trial):
    
    param = {'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
             'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
             'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
             'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
             'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
             'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.009,0.01,0.012,0.014,0.016,0.018, 0.02]),
             'n_estimators': trial.suggest_int("n_estimators", 50, 400),
             'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17,20]),
             'random_state': 42,
             'min_child_weight': trial.suggest_int('min_child_weight', 1, 300)}
            

    model = XGBRegressor(**param)  
    
    model.fit(X_train, y_train,eval_set=[(X_test,y_test)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(X_test)  
    
    return WMAE(X_test ,y_test, preds)

Estudo com intenção de diminuir o WMAE 

study = optuna.create_study(direction= 'minimize')
study.optimize(objective, n_trials= 150)

print('Number of finished trials:', len(study.trials))

In [ ]:
# Melhores parâmetros encontrados pelo optuna = Trial 82 finished with value: 1242.22

params= {'lambda': 1.3455233520686138, 
         'alpha': 0.316230354228637, 
         'colsample_bytree': 0.9, 
         'subsample': 1.0, 
         'learning_rate': 0.014, 
         'n_estimators': 385, 
         'max_depth': 20, 
         'min_child_weight': 8}

In [ ]:
# Rodando o modelo com os melhores parâmetros 

model = XGBRegressor(**params)  
    
model.fit(X_train, y_train)
    
preds = model.predict(X_test)

WMAE(X_test, y_test, preds)

In [ ]:
# teste do WMAE
preds_validation= model.predict(X_valid)

WMAE(X_valid, y_valid, preds_validation)

Tivemos uma queda expressiva no WMAE após o estudo com optuna, saindo de 6413.33 para 1289.05. 

# Predições no dataset de teste

In [ ]:
# Criação da variável X no dataset de submissão do desafio

X_entry= df_test[X.columns]

In [ ]:
# Persisitindo o encoder, previamente salvo (o pickle foi aberto quando persistimos o enconder para X_train, X_test e X_valid), 
# nas variáveis categóricas do X de submissão 

X_entry[encoded_cols_train]= enc.transform(X_entry[encfeat])
X_entry.drop(columns= encfeat, inplace= True)

In [ ]:
# Predição 

preds_entry= model.predict(X_entry)

X_entry["Weekly_Sales"]= preds_entry

In [ ]:
# Verificação das predições X realizado em anos anteriores. 

plt.figure(figsize=(25, 7))

weekly_sales2010 = df_train.loc[df_train['year']==2010].groupby(['week']).agg({'Weekly_Sales': ['mean']})
weekly_sales2011 = df_train.loc[df_train['year']==2011].groupby(['week']).agg({'Weekly_Sales': ['mean']})
weekly_sales2012 = df_train.loc[df_train['year']==2012].groupby(['week']).agg({'Weekly_Sales': ['mean']})
weekly_sales2012_test = X_entry.loc[X_entry['year']==2012].groupby(['week']).agg({'Weekly_Sales': ['mean']})
weekly_sales2013_test = X_entry.loc[X_entry['year']==2013].groupby(['week']).agg({'Weekly_Sales': ['mean']})

sns.lineplot(weekly_sales2010['Weekly_Sales']['mean'].index, weekly_sales2010['Weekly_Sales']['mean'].values, color='gray')
sns.lineplot(weekly_sales2011['Weekly_Sales']['mean'].index, weekly_sales2011['Weekly_Sales']['mean'].values, color='gray')
sns.lineplot(weekly_sales2012['Weekly_Sales']['mean'].index, weekly_sales2012['Weekly_Sales']['mean'].values, color='gray')

sns.lineplot(weekly_sales2012_test['Weekly_Sales']['mean'].index, weekly_sales2012_test['Weekly_Sales']['mean'].values, color='red')
sns.lineplot(weekly_sales2013_test['Weekly_Sales']['mean'].index, weekly_sales2013_test['Weekly_Sales']['mean'].values, color='red')


plt.xticks(np.arange(1, 53, step=1))
plt.legend(['2010', '2011', '2012','2012 test', '2013 test'])
plt.show()

# Criação do dataset de envio

In [ ]:
# Criação do dataset de envio

submission["Weekly_Sales"]= preds_entry
submission.to_csv('submission.csv',index=False)
submission